# Verify the Aruco sensorprocessing

This notebook verifies that the Aruco sensorprocessing detects the markers in the images. 

In [ ]:
import sys
sys.path.append("..")

from exp_run_config import Config
Config.PROJECTNAME = "BerryPicker"

import pprint
import matplotlib.pyplot as plt

from demonstration.demonstration import Demonstration

from sensorprocessing.sp_helper import get_transform_to_sp
from sensorprocessing import sp_aruco
import random
import torch
# Move data to GPU (if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Specify the experiment
experiment = "sensorprocessing_aruco"
run = "aruco_128" 
exp = Config().get_experiment(experiment, run, creation_style="discard-old")
pprint.pprint(exp)
sp = sp_aruco.ArucoSensorProcessing(exp, device)

In [ ]:
transform = get_transform_to_sp(exp)
demos = []
cameras = []
# load the demonstrations specified in the experiment validation data
for val in exp["validation_data"]:
    run, demo_name, camera = val
    exp_demo = Config().get_experiment("demonstration", run)
    demo = Demonstration(exp_demo, demo_name)
    demos.append(demo)
    cameras.append(camera)

# Choose n pictures from the validation set and store them in lists of images and imagefiles
n = 6
demo = demos[0]
camera = cameras[0]
images = []
imagefiles = []
for i in range(demo.metadata["maxsteps"]):
    rnd = random.randint(0, demo.metadata["maxsteps"] - 1)
    imagefiles.append(demo.get_image_path(rnd))
    image, _ = demo.get_image(rnd, device=device, camera=camera, transform=transform)
    images.append(image)


In [ ]:
# Verify whether we can process the images loaded from the demonstration
for image in images:
    z = sp.process(image)
    print(f"The encoding is\n {z}")